In [18]:
import findspark
findspark.init() # this must be executed before the below import

In [19]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [20]:
import time
import numpy as np
from numpy import genfromtxt
import rtree
from rtree import index

In [21]:
sc = SparkContext()
sqlContext = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-4-45c7a0aab248>:1 

In [ ]:
# use a Rtree to index the partitions, the id is the partition id
# use a dictionary to save the {partition_id : parquet_id}
# compose parquet file address with the parquet_id, or do we directly use partition id as parquet id

In [ ]:
def find_overlap_parquets(query, partition_index):
    '''
    find out all the overlap partition ids
    '''
    query_lower = [qr[0] for qr in query]
    query_upper = [qr[1] for qr in query]
    query_border = tuple(query_lower + query_upper)
    overlap_pids = list(partition_index.intersection(query_border))
    
    return overlap_pids

In [ ]:
def get_parquet_file_paths(partition_ids, hdfs_path=None):
    
    if hdfs_path == None:
        hdfs_path = 'hdfs://localhost:9000/user/cloudray/NORA/'
    
    result_paths = []
    
    for pid in partition_ids:
        partition_name = 'partition_' + str(pid)+'.parquet'
        path = hdfs_path + partition_name
        result_paths.append(path)
        
    return result_paths

In [ ]:
def transform_query_to_sql(query, used_dims, column_name_dict):
    sql = ''
    for i, dim in enumerate(used_dims):
        if query[i][0] != -1:
            sql += column_name_dict[dim] + '>=' + str(query[i][0]) + ' and '
        if query[i][1] != -1:
            sql += column_name_dict[dim] + '<=' + str(query[i][1]) + ' and '
    sql = sql[0:-4] # remove the last 'and '
    return sql

In [ ]:
def query_with_parquets(query, partition_index, used_dims, column_name_dict, 
                        hdfs_path, print_execution_time=False):
    '''
    parameters:
    @query: should contains the same dimension that partition_index holds
    
    first, find the overlapped parquet ids
    second, load these parquets
    third, query from the loaded parquets
    '''
    
    pids = find_overlap_parquets(query, partition_index)
    paths = get_parquet_file_paths(pids, hdfs_path)
    
    start_time = time.time()
    
    dfs = sqlContext.read.parquet(*paths)
    dfs.write.mode('overwrite').parquet(hdfs_path+'temp.parquet')
    dfs = sqlContext.read.parquet(hdfs_path+'temp.parquet')
    
    end_time_1 = time.time()
    
    sql = transform_query_to_sql(query, used_dims, column_name_dict)
    
    end_time_2 = time.time()
    
    query_result = dfs.filter(sql).collect()
    
    end_time_3 = time.time()
    
    parquet_load_time = end_time_1 - start_time
    query_translation_time = end_time_2 - end_time_1
    query_execution_time = end_time_3 - end_time_2
    
    if print_execution_time:
        print('parquet loading time: ', parquet_load_time)
        print('query translation time: ', query_translation_time)
        print('query execution time: ', query_execution_time)
    
    return (query_result, len(paths), parquet_load_time, query_translation_time, query_execution_time, dfs.count())

In [22]:
def query_with_parquets_seperate(query, partition_index, used_dims, column_name_dict, 
                        hdfs_path, print_execution_time=False):
    '''
    parameters:
    @query: should contains the same dimension that partition_index holds
    
    first, find the overlapped parquet ids
    second, load these parquets
    third, query from the loaded parquets
    '''
    
    pids = find_overlap_parquets(query, partition_index)
    paths = get_parquet_file_paths(pids, hdfs_path)
    
    start_time = time.time()
    for path in paths:
        dfs = sqlContext.read.parquet(*paths)
        sql = transform_query_to_sql(query, used_dims, column_name_dict)
        query_result = dfs.filter(sql).collect()
    end_time = time.time()
    
    query_time = end_time - start_time
    
    if print_execution_time:
        print('query time: ', query_time)
    
    return ([query_time], len(paths), query_time, query_time, query_time, 1)

In [23]:
def query_from_single_file(filepath, query, used_dims, column_name_dict):
    start_time = time.time()
    dfs = sqlContext.read.csv(filepath, header=False, inferSchema=True)
    end_time_1 = time.time()
    sql = transform_query_to_sql(query, used_dims, column_name_dict)
    end_time_2 = time.time()
    query_result = dfs.filter(sql).collect()
    end_time_3 = time.time()
    parquet_load_time = end_time_1 - start_time
    query_translation_time = end_time_2 - end_time_1
    query_execution_time = end_time_3 - end_time_2
    return (query_result, 1, parquet_load_time, query_translation_time, query_execution_time, dfs.count())

In [24]:
# # = = = Unit Test = = =

# column_name_dict = {0:'_c0', 1:'_c1', 2:'_c2', 3:'_c3'}
# query = [[-1,-1],[10,20],[10,-1]]

# sql = transform_query_to_sql(query, column_name_dict)
# print(sql)

In [25]:
def load_query(path):
    query_set = np.genfromtxt(path, delimiter=' ')
    query_set = query_set.reshape(len(query_set),-1,2)
    return query_set

def kdnode_2_border(kdnode):
    lower = [domain[0] for domain in kdnode[0]]
    upper = [domain[1] for domain in kdnode[0]]
    border = tuple(lower + upper) # non interleave
    return border

def load_partitions_from_file(path):
    '''
    the loaded stretched_kdnodes: [num_dims, l1,l2,...,ln, u1,u2,...,un, size, id, pid, left_child,id, right_child_id]
    '''
    stretched_kdnodes = genfromtxt(path, delimiter=',')
    num_dims = int(stretched_kdnodes[0,0])
    kdnodes = []
    for i in range(len(stretched_kdnodes)):
        domains = [ [stretched_kdnodes[i,k+1],stretched_kdnodes[i,1+num_dims+k]] for k in range(num_dims) ]
        row = [domains]
        row.append(stretched_kdnodes[i,2*num_dims+1])
        # to be compatible with qd-tree's partition, that do not have the last 4 attributes
        if len(stretched_kdnodes[i]) > 2*num_dims+2:
            row.append(stretched_kdnodes[i,-4])
            row.append(stretched_kdnodes[i,-3])
            row.append(stretched_kdnodes[i,-2])
            row.append(stretched_kdnodes[i,-1])
        kdnodes.append(row)
    return kdnodes

def batch_query(queryset, partition_path, used_dims, column_name_dict, hdfs_path):
    
    partitions = load_partitions_from_file(partition_path)
    
    p = index.Property()
    p.leaf_capacity = 100 # cannot be less than 100, indicate the maximum capacity
    p.fill_factor = 0.5
    p.overwrite = True
    
    partition_index = index.Index(properties = p)
    for i in range(len(partitions)):
        # qd-tree do not have this
        #partition_index.insert(int(partitions[i][-4]), kdnode_2_border(partitions[i])) 
        partition_index.insert(i, kdnode_2_border(partitions[i]))
    
    start_time = time.time()
    
    # add statistics result
    results = []
    for query in queryset:
        result = query_with_parquets(query, partition_index, used_dims, column_name_dict, hdfs_path)
        #result = query_with_parquets_seperate(query, partition_index, used_dims, column_name_dict, hdfs_path)
        #result = query_from_single_file('/home/cloudray/Downloads/TPCH_12M_8Field.csv', query, used_dims, 
                                        #column_name_dict)
        results.append(result)
    
    end_time = time.time()
    
    total_records = 0
    total_partitions = 0
    total_loading_time = 0
    total_translation_time = 0
    total_execution_time = 0
    total_candidates = 0
    for i in range(len(results)):
        total_records += len(results[i][0])
        total_partitions += results[i][1]
        total_loading_time += results[i][2]
        total_translation_time += results[i][3]
        total_execution_time += results[i][4]
        total_candidates += results[i][5]
    
    query_response_time = end_time - start_time
    avg_query_response_time = query_response_time / len(queryset)
    
#     print('total query response time: ', query_response_time)
#     print('average query response time: ', avg_query_response_time)
    
    avg_records = total_records / len(queryset)
    avg_partitions = total_partitions / len(queryset)
    avg_loading_time = total_loading_time / len(queryset)
    avg_translation_time = total_translation_time / len(queryset)
    avg_execution_time = total_execution_time / len(queryset)
    avg_candidates = total_candidates / len(queryset)
    
    print('total records: ', total_records)
    print('total partitions: ', total_partitions)
    print('total loading time: ', total_loading_time)
    print('total translation time: ', total_translation_time)
    print('total execution time: ', total_execution_time)
    print('total candidate: ', total_candidates)
    
    print('average records: ', avg_records)
    print('average partitions: ', avg_partitions)
    print('average loading time: ', avg_loading_time)
    print('average translation time: ', avg_translation_time)
    print('average execution time: ', avg_execution_time)
    print('average candidate: ', avg_candidates)

In [28]:
# = = = Execution = = =

query_path1 = '/home/cloudray/NORA_Query/intro_distribution.csv'
query_path2 = '/home/cloudray/NORA_Query/intro_random.csv'
queryset1 = load_query(query_path1)
queryset2 = load_query(query_path2)

# queryset = np.concatenate((queryset1[0:40], queryset2[0:10]), axis=0)
queryset = np.concatenate((queryset1[40:], queryset2[10:]), axis=0)

# partition_path = '/home/cloudray/NORA_Partitions/nora_partitions'
partition_path = '/home/cloudray/NORA_Partitions/qd_tree_partitions'
used_dims = [1,2]
num_dims = 8
column_names = ['_c'+str(i) for i in range(num_dims)]
column_name_dict = {}
for i in range(num_dims):
    column_name_dict[i] = column_names[i]

# hdfs_path = 'hdfs://localhost:9000/user/cloudray/NORA/merged/'
hdfs_path = 'hdfs://localhost:9000/user/cloudray/QdTree/merged/'


batch_query(queryset, partition_path, used_dims, column_name_dict, hdfs_path)

total records:  1269797
total partitions:  74
total loading time:  25.816479444503784
total translation time:  0.0015883445739746094
total execution time:  13.516983270645142
total candidate:  14376071
average records:  25395.94
average partitions:  1.48
average loading time:  0.5163295888900756
average translation time:  3.176689147949219e-05
average execution time:  0.27033966541290283
average candidate:  287521.42


In [ ]:
# total records:  1269797
# total partitions:  463
# total loading time:  33.078574419021606
# total translation time:  0.001646280288696289
# total execution time:  13.950875759124756
# total candidate:  3026807
# average records:  25395.94
# average partitions:  9.26
# average loading time:  0.6615714883804321
# average translation time:  3.292560577392578e-05
# average execution time:  0.2790175151824951
# average candidate:  60536.14
    
# total records:  1269797
# total partitions:  74
# total loading time:  25.816479444503784
# total translation time:  0.0015883445739746094
# total execution time:  13.516983270645142
# total candidate:  14376071
# average records:  25395.94
# average partitions:  1.48
# average loading time:  0.5163295888900756
# average translation time:  3.176689147949219e-05
# average execution time:  0.27033966541290283
# average candidate:  287521.42

In [ ]:
# queryset = [[[1000, 9000],[114.4, 185.3]]]

# # partition_path = '/home/cloudray/NORA_Partitions/nora_test' # 1.06s
# partition_path = '/home/cloudray/NORA_Partitions/qd_tree_test' # 1.39s

# # hdfs_path = 'hdfs://localhost:9000/user/cloudray/NORA_Test/merged/'
# hdfs_path = 'hdfs://localhost:9000/user/cloudray/QdTree_Test/merged/'

# used_dims = [1,2]
# num_dims = 8
# column_names = ['_c'+str(i) for i in range(num_dims)]
# column_name_dict = {}
# for i in range(num_dims):
#     column_name_dict[i] = column_names[i]

# batch_query(queryset, partition_path, used_dims, column_name_dict, hdfs_path)

In [ ]:
# # nora
# total query response time:  29.44448232650757
# average query response time:  0.5888896465301514

# total records: 1269797
# total partitions:  463
# total loading time:  2.648822784423828
# total translation time:  0.001538991928100586
# total execution time:  24.893444061279297
# total candidate:  3026807

# average records:  25395.94
# average partitions:  9.26
# average loading time:  0.05297645568847656
# average translation time:  3.077983856201172e-05
# average execution time:  0.4978688812255859
# average candidate:  60536.14


# # qd-tree
# total query response time:  18.707958221435547
# average query response time:  0.37415916442871094

# total records:  1269797
# total partitions:  74
# total loading time:  1.9194436073303223
# total translation time:  0.001483917236328125
# total execution time:  12.638132810592651
# total candidate:  14376071

# average records:  25395.94
# average partitions:  1.48
# average loading time:  0.03838887214660645
# average translation time:  2.96783447265625e-05
# average execution time:  0.252762656211853
# average candidate:  287521.42


# # single file (single partition)
# total records:  1269797
# total partitions:  50
# total loading time:  548.5099625587463
# total translation time:  0.0016028881072998047
# total execution time:  249.31028032302856
# total candidate:  599899800

# average records:  25395.94
# average partitions:  1.0
# average loading time:  10.970199251174927
# average translation time:  3.2057762145996096e-05
# average execution time:  4.9862056064605715
# average candidate:  11997996.0

# # execute query directly on each parquet file
# total query response time:  330.26623010635376
# average query response time:  6.605324602127075
    
# # try save the loaded dataframe into a new single parquet file
# # load and execute query on it
# total loading time:  35.37981963157654
# total execution time:  10.4388906955719
    
# average loading time:  0.7075963926315307
# average execution time:  0.208777813911438